In [1]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

oai_client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

df=pd.read_pickle('cleaned_quests_with_glossary.pkl')

In [2]:
TRANSLATION_SYSTEM_PROMPT = """You are a professional translator specializing in MMORPG game localization, responsible for translating World of Warcraft content from English to Ukrainian. 
You will receive a JSON file containing an "original" section with the object to translate. 
Your task is to respond with a JSON containing single section, "translation", that replicates the structure of the "original" section, replacing source text with translated values while maintaining all keys.
Use your knowledge of the Warcraft universe to ensure accuracy and immersion.

### **Guidelines**:

1. **Preserve Structure**:
   - The output JSON must retain the exact structure of the "original" section.
   - Ensure all keys remain the same while replacing values with their Ukrainian translations.

2. **Maintain Placeholders**:
   - Always replace name/race/class placeholders in the original text with curly-bracedanslated placeholders with shortened Ukrainian grammatical case after a semicolon. For `<name>`, `<race>`, `<class>` use `{ім'я:н}`, `{раса:д}`, `{клас:к}` with correct grammatical case afet semicolon.

3. **Adapt for Gender**:
   - Translate text to be suitable for both genders. For placeholders with gender-specific variations (e.g., `<his/her>`, `<priest/priestess>`) and translated words that have different gender forms use the format `{стать:male:female}` (e.g., `{стать:його:її}`, `{стать:жрець:жриця}`).
   - Gender placeholders should contain full words, so don't break words with placeholder: for example, instead of кмітлив{стать:ий:а} use {стать:кмітливий:кмітлива}
   - Gender placeholders can contain more that one word. If there's several consequent gender-specific placeholders or they are divided with small word - put tham in single placeholder. Fo example: instead of "дуже {стать:достойний:достойна} та {стать:спроможний:спроможна}" use "дуже {стать:достойний та спроможний:достойна та спроможна}"

4. **Formal and Informal Tone**:
   - **Quest `objective` field**: Translate using a formal tone, where "you" becomes "ви" and "your" becomes "ваш".
   - **Quest `description`, `progress`, and `completion` fields**: By default, translate these fields using an informal tone, where "you" becomes "ти" and "your" becomes "твій.". If text describes environment/interaction or NPC speaks to player politely using "sir", "lady" etc - use formal tone.

5. **Preserve Descriptive Formatting**:
   - Retain all text enclosed in angle brackets (`<>`) that describes the environment, character reactions, or similar narrative elements. Translate the content within the brackets but preserve their formatting.

6. **Maintain Lore and Style**:
   - Ensure the translation captures the epic, immersive tone of World of Warcraft.
   - Use established Warcraft terminology and lore-consistent language in Ukrainian.

7. **Glossaries and terminology**:
   - Always strictly refer to the translation glossary. Always use terms' translations from glossary, even if other more commonly used or recognized translations are available.
   - Use consistent terminology throughout all translations. Adhere strictly to any provided glossaries, ensuring uniformity in translating terms such as character names, locations, and game-specific items.

8. **Readability**:
   - Ensure translations are natural, engaging, and free from awkward phrasing.
   - Text should feel native and immersive for Ukrainian-speaking players.

---

### **Input Example**:

```json
{
  "original": {
    "quest": {
      "title": "Get rid of these boars",
      "objective": "Kill 12 boars and return to John at Scarlet Monastery in Silverpine Forest.",
      "description": "Reports have come in that wild boars trample our crops. We thought that other adventurer had dealt with this, <class>!\n\nI suggest that you to teach them a lesson as proof of your devotion.",
      "progress": "Have you clear our fields of wild boars?",
      "completion": "<John nods to you.>\n\nWe are pleased that you have prooved your devition, <name>. Surely now that you have made our point clear multiple times, their senseless attacks upon our crops will cease?"
    }
  }
}
```

---

### **Output Example**:

```json
{
  "translation": {
    "quest": {
      "title": "Позбудься цих вепрів",
      "objective": "Вбийте 12 вепрів і поверніться до Джона в Багряний Монастир, що в Срібнохвойному пралісі.",
      "description": "Надходять повідомлення про диких вепрів, що топчуть наші посіви. Ми думали, що інші шукачі пригод вже розібралися з цим, {клас:к}!\n\nРаджу тобі провчити їх як доказ твоєї відданості.",
      "progress": "Ти вже {стать:очистив:очистила} наші поля від диких вепрів?",
      "completion": "<Джон киває вам.>\n\nМи задоволені, що ти {стать:довів:довела} свою відданість, {ім'я:к}. Тепер, коли ми вже неодноразово довели свою точку зору, може їхні безглузді напади на наші посіви припиняться?"
    }
  }
}
```

---
"""

In [3]:
def generate_request_json(row):
    return {
        "original": {
            "quest": {
                "title": row["title_en"],
                "objective": row["objective_en"],
                "description": row["description_en"],
                "progress": row["progress_en"],
                "completion": row["completion_en"]
            }
        },
        "glossary": {
            "glossary": row["glossary"]
        }
    }

def translate_row(row):
    import json
    messages = [{'role': 'system', 'content': TRANSLATION_SYSTEM_PROMPT}, {'role': 'user', 'content': str(generate_request_json(row))}]
    response = oai_client.chat.completions.create(
    # model="ft:gpt-4o-2024-08-06:personal:classicua-v5:AXuONZ0V",
    model="ft:gpt-4o-2024-08-06:personal:classicua-v6:AeQq2YLn",
    messages=messages,
    response_format={"type":"json_object"},
    temperature=0.7,
    top_p=0.7
    )
    json_response = response.choices[0].message.content
    translation = json.loads(json_response)['translation']['quest']
    print(translation)
    return {
            "title_uk": translation.get("title", None),
            "objective_uk": translation.get("objective", None),
            "description_uk": translation.get("description", None),
            "progress_uk": translation.get("progress", None),
            "completion_uk": translation.get("completion", None),
    }


In [4]:
translated = df[df['title_uk'].notnull()]
untranslated = df[df['title_uk'].isnull()]

[{'en': 'The Alliance', 'uk': 'Альянс'},
 {'en': 'Squire Cuthbert', 'uk': 'зброєносець Катберт'}]

In [5]:
translate_row(df[df['id']==83936].iloc[0])

{'title': 'Завдання Долтона', 'objective': 'Поговоріть зі зброєносцем Катбертом, щоб дізнатися більше про завдання сера Долтона.', 'description': "<Зброєносець Катберт не відводить очей від місця, де горить тіло Долтона. Він явно перебуває в шоці. Видно, що він не в змозі продовжувати виконувати завдання, яке вони з його господарем намагалися завершити. Можливо, тобі варто поговорити з ним і з'ясувати, чи можеш ти чимось допомогти.>", 'progress': 'Я... дякую тобі.', 'completion': 'Я... не знаю, що сказати. Попри всі ці обставини, я маю за честь служити {стать:іншому істинному герою:іншій істинній героїні} Альянсу. Я можу лише сподіватися, що служитиму тобі краще, ніж служив серу Долтону.\n\nВеди, {стать:герою:героїне}.'}


{'title_uk': 'Завдання Долтона',
 'objective_uk': 'Поговоріть зі зброєносцем Катбертом, щоб дізнатися більше про завдання сера Долтона.',
 'description_uk': "<Зброєносець Катберт не відводить очей від місця, де горить тіло Долтона. Він явно перебуває в шоці. Видно, що він не в змозі продовжувати виконувати завдання, яке вони з його господарем намагалися завершити. Можливо, тобі варто поговорити з ним і з'ясувати, чи можеш ти чимось допомогти.>",
 'progress_uk': 'Я... дякую тобі.',
 'completion_uk': 'Я... не знаю, що сказати. Попри всі ці обставини, я маю за честь служити {стать:іншому істинному герою:іншій істинній героїні} Альянсу. Я можу лише сподіватися, що служитиму тобі краще, ніж служив серу Долтону.\n\nВеди, {стать:герою:героїне}.'}

In [37]:
# filtered_df = untranslated[(untranslated['expansion']=='tbc') & (untranslated['cat']=='Outland/Terokkar Forest')]
filtered_df = untranslated[(untranslated['expansion']=='sod')]
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110 entries, 5767 to 7058
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              110 non-null    int64 
 1   expansion       110 non-null    object
 2   title_en        110 non-null    object
 3   objective_en    109 non-null    object
 4   description_en  109 non-null    object
 5   progress_en     67 non-null     object
 6   completion_en   102 non-null    object
 7   title_uk        0 non-null      object
 8   objective_uk    0 non-null      object
 9   description_uk  0 non-null      object
 10  progress_uk     0 non-null      object
 11  completion_uk   0 non-null      object
 12  cat             110 non-null    object
 13  glossary        110 non-null    object
dtypes: int64(1), object(13)
memory usage: 12.9+ KB


In [38]:
translation = filtered_df.apply(translate_row, axis=1)
translation_df = pd.DataFrame(translation.tolist(), index=filtered_df.index)

untranslated.update(translation_df)

{'title': 'Нездатний лідер', 'objective': 'Джорін Мертвий Зір просить вас знайти Кілрата і поговорити з ним.', 'description': 'Я втік від неминучої смерті, переживши боягузливий нічний напад брилоруких огрів на моє село.\n\nАле скажи мені, {стать:незнайомцю:незнайомко}, для чого?\n\n<Джорін вказує на Гарроша.>\n\nВін відмовився допомогти нам. Огрі розгромили моє місто, а він сидить тут, плачучи в вогонь. Що з нами буде?\n\nЯ послав розвідника, Кілрата, в село, тепер вже в руїни Кривавої Зіниці. Чи зможеш ти знайти його та допомогти йому? Він ховається на кордоні Наґранда і Терокару.', 'progress': None, 'completion': '<Кілрат говорить тихим голосом.>\n\nЦі огри надзвичайно цікаві. І надзвичайно дурні. Подивися, як цей товстий змушує танцювати інших, еее, менш товстих. Це просто заворожує.'}
{'title': 'Не вбивати товстого', 'objective': "Кілрат з лісу Тероккар просить вас вбити 10 загарбників з клану Кам'яного Кулака і побити безжального Ункора, поки він не заговорить.", 'description': '

In [52]:
untranslated[untranslated['title_uk'].notnull()]

,id,expansion,title_en,objective_en,description_en,progress_en,completion_en,title_uk,objective_uk,description_uk,progress_uk,completion_uk,cat,glossary
5767,9888,tbc,The Impotent Leader,Jorin Deadeye has asked that you find and spea...,"I have escaped certain death, surviving a cowa...",None,<Kilrath speaks in a hushed tone.>\n\nThese og...,Нездатний лідер,Джорін Мертвий Зір просить вас знайти Кілрата ...,"Я втік від неминучої смерті, переживши боягузл...",None,<Кілрат говорить тихим голосом.>\n\nЦі огри на...,Outland/Terokkar Forest,"[{'en': 'ogre', 'uk': 'огр'}, {'en': 'The Blee..."
5768,9889,tbc,Don't Kill the Fat One,Kilrath in Terokkar Forest has asked that you ...,I have been watching these beasts for days. Th...,I submit! I submit!,Let me live and I'll tell you whatever you wan...,Не вбивати товстого,Кілрат з лісу Тероккар просить вас вбити 10 за...,Я кілька днів стежив за цими потворами. Товсти...,Я здаюся! Я здаюся!,"Дай мені жити, і я скажу все, що ти хочеш знати!",Outland/Terokkar Forest,"[{'en': 'Terokkar Forest', 'uk': 'ліс Тероккар'}]"
5769,9890,tbc,Success!,Take the information that you beat out of Unko...,"For many years, Boulderfist ruled all ogres in...",None,"Slow down, <name>. I am but a lowly rogue. I h...",Успіх!,Вибийте інформацію з Ункора та поверніться до ...,Багато років Кам'яний Кулак правив усіма ограм...,,"Заспокойся, {ім'я:к}. Я лише простий пройдисві...",Outland/Terokkar Forest,"[{'en': 'ogre', 'uk': 'огр'}, {'en': 'rogue', ..."
5819,9951,tbc,It's Watching You!,Kill Naphthal'ar and then return to Warden Tre...,"Quick, hide! The eyes of Naphthal'ar are upon ...",We should never have invited them to dinner. T...,"It's for the best, really. If it cannot civill...",Воно слідкує за тобою!,"Убийте Нафтала'ра, а потім поверніться до варт...","Швидко, ховайся! Очі Нафтала'ра дивляться на т...",Не треба було запрошувати їх на вечерю. Це вже...,"Так буде краще, чесно. Якщо воно не може по-лю...",Outland/Terokkar Forest,"[{'en': 'Terokkar Forest', 'uk': 'ліс Тероккар..."
5823,9957,tbc,What's Wrong at Cenarion Thicket?,Speak with one of the druids at the Cenarion T...,We haven't heard from the druids at the Cenari...,None,I'm glad that the refuge sent you. The druids ...,Що сталося в Кенарійській гущавині?,Поговоріть з одним з друїдів Кенарійської гуща...,Ми вже досить давно не отримували звісток від ...,None,"Добре, що притулок {стать:надіслав:надіслала} ...",Outland/Terokkar Forest,"[{'en': 'druid', 'uk': 'друїд/друїдка'}, {'en'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6849,11074,tbc,Tokens of the Descendants,Collect Time-Lost Scrolls from the time-lost a...,The descendants held prisoner by the Skettis t...,I require a token from each of the descendants...,Excellent! Take this. I've prepared this bundl...,Знаки нащадків,"Зберіть сувої, загублені в часі, у араккоа, за...","В цій книзі записані нащадки, які перебувають ...",Мені потрібен знак від кожного з нащадків найм...,Чудово! Візьми це. Я підготував цей згорток з ...,Outland/Terokkar Forest,"[{'en': 'Skettis', 'uk': 'Скеттіс'}]"
7050,11505,tbc,Spirits of Auchindoun,Help the Alliance secure a tower in the Bone W...,Auchindoun's spirits have no stake in the conf...,"The spirits are restless, <name>. Have you acc...",The spirits are one step closer to resting in ...,Духи Акіндуна,Допоможіть Альянсу захопити вежу на Кістяній п...,Духи Акіндуна не зацікавлені в конфлікті між А...,"Духи не знаходять собі спокою, {ім'я:к}. Ти {с...",Духи стали на крок ближчими до мирного спочинк...,Outland/Terokkar Forest,"[{'en': 'The Horde', 'uk': 'Орда'}, {'en': 'Th..."
7051,11506,tbc,Spirits of Auchindoun,Help the Horde secure a Spirit Tower in the Bo...,"The draenei might be our enemies, but the spir...",The spirits of Auchindoun are all around us. W...,The spirits are closer to finding peace and ha...,Духи Акіндуна,Допоможіть Орді захопити вежу духів на Кістяні...,"Дренеї можуть бути нашими ворогами, але духи ї...",Духи Акіндуна навколо нас. Чи да

In [53]:
untranslated[untranslated['title_uk'].notnull()].to_pickle('translations_df.pkl')

In [54]:
translations_df = pd.read_pickle('translations_df.pkl')

In [55]:
from crowdin_api import CrowdinClient

CROWDIN_PROJECT_ID=393919
crowdin_client = CrowdinClient(token=os.getenv('CROWDIN_TOKEN'), project_id=CROWDIN_PROJECT_ID, )

In [5]:
crowdin_files = dict()
offset = 0
page_size = 500
while (True):
    files = crowdin_client.source_files.list_files(CROWDIN_PROJECT_ID, offset=offset, limit=page_size)
    if not files['data']:
        break
    for file in files['data']:
        file_id = file['data']['id']
        file_path = file['data']['path']
        crowdin_files[file_path] = file_id
    offset += page_size

crowdin_files

{'/quests/Battlegrounds/Alterac Valley/A Gallon of Blood_7385.xml': 8826,
 '/quests/Battlegrounds/Alterac Valley/Ally of the Tauren_7362.xml': 8830,
 '/quests/Battlegrounds/Alterac Valley/Alterac Valley Graveyards_7081.xml': 8834,
 '/quests/Battlegrounds/Alterac Valley/Armor Scraps_7223.xml': 8838,
 '/quests/Battlegrounds/Alterac Valley/Begin the Attack_6846.xml': 8840,
 '/quests/Battlegrounds/Alterac Valley/Brotherly Love_7281.xml': 8842,
 '/quests/Battlegrounds/Alterac Valley/Brotherly Love_7282.xml': 8844,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Guses Fleet_6825.xml': 8846,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Ichmans Fleet_6943.xml': 8848,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Jeztors Fleet_6826.xml': 8850,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Mulvericks Fleet_6827.xml': 8852,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Slidores Fleet_6942.xml': 8854,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Vipores Fl

In [56]:
def get_quest_filename(row):
    quest_id = row['id']
    quest_title = row['title_en']
    valid_chars = frozenset('-.() abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    return ''.join(c for c in quest_title if c in valid_chars) + '_' + str(quest_id)

def get_path_for_quest(row):
    path = '/quests' + (f"_{row['expansion']}" if row['expansion'] != 'classic' else '')
    filename = get_quest_filename(row)
    path = path + f"/{row['cat']}/{filename}.xml"
    return path

def upload_translation(string_data, translation_text):
    crowdin_client.source_strings.get_string(string_data['id'])
    try:
        crowdin_client.string_translations.add_translation(stringId=string_data['id'], languageId='uk', text=translation_text)
        print(f"Uploaded translation for string#{string_data['id']}")
    except:
        print(f"Error uploading translation for string#{string_data['id']}")


def upload_translations(row):
    path = get_path_for_quest(row)
    file_id = crowdin_files.get(path)
    if not file_id:
        print(f"File #{file_id} not found for {path}.")
        return
    strings = crowdin_client.source_strings.list_strings(fileId=file_id)['data']
    strings_by_key = {string_data['data']['identifier']: string_data['data'] for string_data in strings}
    upload_translation(strings_by_key['TITLE'], row['title_uk'])
    if strings_by_key.get('OBJECTIVE') and row['objective_uk']:
        upload_translation(strings_by_key['OBJECTIVE'], row['objective_uk'])
    if strings_by_key.get('DESCRIPTION') and row['description_uk']:
        upload_translation(strings_by_key['DESCRIPTION'], row['description_uk'])
    if strings_by_key.get('PROGRESS') and row['progress_uk']:
        upload_translation(strings_by_key['PROGRESS'], row['progress_uk'])
    if strings_by_key.get('COMPLETION') and row['completion_uk']:
        upload_translation(strings_by_key['COMPLETION'], row['completion_uk'])



In [57]:
len(list(translations_df.iterrows()))

110

In [58]:
for _, row in translations_df.iterrows():
    filename = get_quest_filename(row)
    try:
        upload_translations(row)
        print(f"Uploaded translations for {filename}")
    except Exception as e:
        print(f"Error uploading translations for {filename}: {e}")

Uploaded translation for string#66434
Uploaded translation for string#66436
Uploaded translation for string#66438
Uploaded translation for string#66440
Uploaded translations for The Impotent Leader_9888
Uploaded translation for string#62206
Uploaded translation for string#62208
Uploaded translation for string#62210
Uploaded translation for string#62212
Uploaded translation for string#62214
Uploaded translations for Dont Kill the Fat One_9889
Error uploading translation for string#66140
Uploaded translation for string#66142
Uploaded translation for string#66144
Uploaded translation for string#66146
Uploaded translations for Success_9890
Uploaded translation for string#65542
Uploaded translation for string#65544
Uploaded translation for string#65546
Uploaded translation for string#65548
Uploaded translation for string#65550
Uploaded translations for Its Watching You_9951
Error uploading translation for string#65346
Uploaded translation for string#65348
Uploaded translation for string#653

KeyboardInterrupt: 